In [4]:
#Simulations and Benchmarking Comparison with LASSO
import numpy as np
from numpy.linalg import cholesky
from sklearn.linear_model import *
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from dataclasses import dataclass
import cupy as cp
import numpy as np
import random

# ---------------------- Config ----------------------
N = 500            # samples
P = 1000           # total features
S = 200            # number of true nonzero features ("top 200 X_i")
BLOCK = 50         # correlation block size
RHO = 0.8          # within-block AR(1) correlation
SNR = 16.0          # signal-to-noise ratio: Var(Xb)/Var(eps)
NUM_RUNS = 5       # repeat the simulation
RANDOM_200 = False # if True, choose 200 random causal features; else first 200
SEED = 1234

# ------------------- Simulation utils ----------------
def make_block_ar1_X(n, p, block=50, rho=0.6, rng=None):
    """
    Generate X with blockwise AR(1) correlation across features.
    Each block has covariance Sigma_ij = rho^{|i-j|}.
    """
    if rng is None:
        rng = np.random.default_rng()
    X = np.zeros((n, p), dtype=float)
    num_blocks = int(np.ceil(p / block))
    for b in range(num_blocks):
        s = b * block
        e = min((b + 1) * block, p)
        m = e - s
        # AR(1) covariance and Cholesky
        idx = np.arange(m)
        Sigma = rho ** np.abs(idx[:, None] - idx[None, :])
        L = cholesky(Sigma + 1e-8 * np.eye(m))
        Z = rng.standard_normal(size=(n, m))
        X[:, s:e] = Z @ L.T
    # standardize columns to mean 0, var 1
    X -= X.mean(axis=0, keepdims=True)
    std = X.std(axis=0, ddof=1, keepdims=True)
    std[std == 0] = 1.0
    X /= std
    return X

def simulate_linear(X, s=200, snr=8.0, random_200=False, rng=None):
    """
    Choose s causal features, draw coefficients, create y with target SNR.
    Returns: y, beta_true, support (boolean mask).
    """
    if rng is None:
        rng = np.random.default_rng()
    n, p = X.shape
    support = np.zeros(p, dtype=bool)
    if random_200:
        idx = rng.choice(p, size=s, replace=False)
    else:
        idx = np.arange(s)  # "top 200" = first 200 columns
    support[idx] = True

    beta = np.zeros(p, dtype=float)
    # effect sizes: random signs & magnitudes
    beta[idx] = rng.choice([-1, 1], size=s) * (0.5 + rng.random(s))

    y_signal = X @ beta
    var_signal = y_signal.var()
    var_noise = var_signal / snr if snr > 0 else 1.0
    eps = rng.standard_normal(n) * np.sqrt(var_noise)
    y = y_signal + eps
    # standardize y (optional; LASSO pipeline will standardize anyway)
    return y, beta, support

def fit_lasso_select(X, y, seed=0):
    """
    LASSO with CV; return indices selected (nonzero coefficients).
    """
    pipe = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        LassoCV(cv=5, alphas=100, random_state=seed, max_iter=10000)
    )
    pipe.fit(X, y)
    coef = pipe.named_steps['lassocv'].coef_
    selected = np.flatnonzero(np.abs(coef) > 1e-10)
    return selected, coef

@dataclass
class Metrics:
    tp: int; fp: int; fn: int; tn: int
    tpr: float; fpr: float
    selected: int

def compute_tpr_fpr(selected_idx, support_mask, p_total):
    """
    TPR = TP / (#true); FPR = FP / (#null).
    """
    selected_mask = np.zeros(p_total, dtype=bool)
    selected_mask[selected_idx] = True

    tp = int(np.sum(selected_mask & support_mask))
    fp = int(np.sum(selected_mask & ~support_mask))
    fn = int(np.sum(~selected_mask & support_mask))
    tn = int(np.sum(~selected_mask & ~support_mask))

    s = int(np.sum(support_mask))
    nulls = p_total - s
    tpr = tp / s if s > 0 else np.nan
    fpr = fp / nulls if nulls > 0 else np.nan
    return Metrics(tp, fp, fn, tn, tpr, fpr, int(np.sum(selected_mask)))


In [5]:
#Our methods
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    cp.random.seed(seed)
set_seed(123)

class fixed_variable_selection:
    def __init__(self,data,prior):
      self.data=data
      self.p_lambda=prior[0]
      self.p_wishart=prior[1]
      self.p_probability=prior[2]

    def dot(self,A,B):
      C=cp.einsum('ijk,ikp->ijp', A, B)
      return C

    def quantity(self,data):
      X,Y=data
      N,P=X.shape
      D=Y.shape[1]
      #Q1: sum n of xni^2 P*1
      #Q2: sum of Yn*Yn^T D*D
      #Q3: sum n of xni*Yn D*P
      #Q4: sum of n xni*xnj P*P

      Q1=cp.sum(X*X,axis=0).reshape(P,1)
      Q2=cp.sum(Y.reshape(N,D,1)*Y.reshape(N,1,D),axis=0)
      Q3=cp.sum(X.reshape(N,1,P)*Y.reshape(N,D,1),axis=0)
      for i in range(N):
          if i==0:
                Q4=X[i].reshape(P,1)*X[i].reshape(1,P)
          else:
            Q4+=X[i].reshape(P,1)*X[i].reshape(1,P)
      return [Q1,Q2,Q3,Q4]


    def VI_w(self,data,E_lambda,E_w_p,E_z,p_lambda,Q):
      X,Y=data
      N,P=X.shape
      D=Y.shape[1]
      #X:N*P
      #Y:N*D
      #E_lambda:D*D
      #E_w_p:D*P
      #E_z:P*2
      #p_lambda[0]:D*D (identity matrix)

      #precision:P*D*D
      precision=Q[0].reshape(P,1,1)*E_lambda+E_z[:,0].reshape(P,1,1)*p_lambda[0]+E_z[:,1].reshape(P,1,1)*p_lambda[1]
      inv_precision=cp.linalg.inv(precision+1e-11)

      mu_i1=X.reshape(N,1,P)*E_w_p #N*D*P
      mu_inter=cp.sum(X.reshape(N,1,P)*(cp.sum(mu_i1,axis=-1,keepdims=True)-mu_i1),axis=0)-Q[2] #D*P
      mu_inter2=-cp.sum(E_lambda*cp.transpose(mu_inter).reshape(P,1,D),axis=-1).reshape(P,1,D) #P*D
      mu=cp.sum(inv_precision*mu_inter2,axis=-1) #P*D

      def Gaussian_expectation(mean,precision,inv_precision):
        E_w=mean
        E_w_2=mean.reshape(P,D,1)*mean.reshape(P,1,D)+inv_precision
        return [cp.transpose(E_w),E_w_2,mean,inv_precision] #D*P, P*D*D

      return Gaussian_expectation(mu,precision,inv_precision)


    def VI_z(self,E_w_2,p_lambda,p):
      #E_w_2:P*D*D
      #p_lambda[0]:D*D
      D=p_lambda[0].shape[0]
      #p:constant 0.5
      p_lambda1=cp.repeat(p_lambda[0].reshape(1,D,D),len(E_w_2),axis=0)
      p_lambda2=cp.repeat(p_lambda[1].reshape(1,D,D),len(E_w_2),axis=0)
     # print(E_w_2)

      z_1=0.5*cp.log(cp.linalg.det(p_lambda[0]))-0.5*cp.trace(self.dot(p_lambda1,E_w_2),axis1=-2, axis2=-1).reshape(-1,1)+cp.log(p+1e-20) #P*1
     # print(z_1)
      z_2=0.5*cp.log(cp.linalg.det(p_lambda[1]))-0.5*cp.trace(self.dot(p_lambda2,E_w_2),axis1=-2, axis2=-1).reshape(-1,1)+cp.log(1-p+1e-20) #P*1

      z=1/(1+cp.exp(z_2-z_1))
      #print(z[0])
      return cp.concatenate([z,1-z],axis=1)

    def VI_lambda(self,data,E_w,inv_precision,p_v,p_V,Q):
      X,Y=data
      N,P=X.shape
      D=Y.shape[1]

      E_w_t=cp.transpose(E_w)
      #E_w_t:P*D
      #inv_precision:P*D*D
      #p_v:constant
      #p_V:D*D

      new_v=p_v+N
      new_V_i=cp.linalg.inv(p_V)+Q[1]-cp.sum(self.dot(E_w_t.reshape(P,D,1),cp.transpose(Q[2]).reshape(P,1,D)),axis=0)\
        -cp.sum(self.dot(cp.transpose(Q[2]).reshape(P,D,1),E_w_t.reshape(P,1,D)),axis=0) #D*D

      new_V_i1=E_w_t.reshape(P,1,D,1)*E_w_t.reshape(1,P,1,D) #P*P*D*D
      for i in range(P):
        new_V_i1[i,i,:,:]+=inv_precision[i]

      new_V_i2=cp.sum(Q[3].reshape(P,P,1,1)*new_V_i1,axis=[0,1])#D*D

      new_V=(new_V_i2+cp.transpose(new_V_i2))/2+new_V_i

      def wishart_expectation(v,V):
        return [v*cp.linalg.inv(V),v,cp.linalg.inv(V)]

      return wishart_expectation(new_v,new_V)

    def variational_inference(self,max_iter,last_run,activate=True):
      if len(last_run)==0:

        def initialization():

          from sklearn import linear_model
          regr = linear_model.LinearRegression()
          regr.fit(cp.asnumpy(self.data[0]),cp.asnumpy(self.data[1]))
          regr.coef_
          i_E_w=cp.array(regr.coef_) #D*P
        #  i_E_w=cp.concatenate([beta.reshape(1,-1),cp.zeros((750,1)).reshape(1,-1)],axis=1).reshape(1,P+1)
          i_E_w_2=cp.transpose(i_E_w).reshape(-1,self.data[1].shape[1],1)*cp.transpose(i_E_w).reshape(-1,1,self.data[1].shape[1]) #P*D*D

          return i_E_w,i_E_w_2,cp.zeros_like(i_E_w_2) #inv_precision

        E_w,E_w_2,inv_precision=initialization()
        self.Q=self.quantity(self.data)

      else:

        E_w,E_w_2,inv_precision,self.p_probability,self.p_wishart=last_run

      for i in range(max_iter):
        E_lambda,c_v,c_V=self.VI_lambda(self.data,E_w,inv_precision,self.p_wishart[0],self.p_wishart[1],self.Q) #P*D*D

        E_z=self.VI_z(E_w_2,self.p_lambda,self.p_probability) #P*2
        c_E_w,E_w_2,c_mu,inv_precision=self.VI_w(self.data,E_lambda,E_w,E_z,self.p_lambda,self.Q)

        E_w=c_E_w #P*D

        Stop=self.early_stop(E_z,i,1e-5,activate)

        if Stop:
            return [E_z,E_w,E_lambda],[E_w,E_w_2,inv_precision,E_z[:,0].reshape(-1,1),[c_v,c_V]]
        if i==max_iter-1:
            return [E_z,E_w,E_lambda],[E_w,E_w_2,inv_precision,E_z[:,0].reshape(-1,1),[c_v,c_V]]


    def sampling_batch(self,iter_s,inner_iter,sub_set,activate=True):

        import random
        for i in range(iter_s):
            if i ==0:
              def initialization():
                  from sklearn import linear_model
                  regr = linear_model.LinearRegression()
                  regr.fit(cp.asnumpy(self.data[0]),cp.asnumpy(self.data[1]))
                  regr.coef_
                  i_E_w=cp.array(regr.coef_) #D*P
                #  i_E_w=cp.concatenate([beta.reshape(1,-1),cp.zeros((750,1)).reshape(1,-1)],axis=1).reshape(1,P+1)
                  i_E_w_2=cp.transpose(i_E_w).reshape(-1,self.data[1].shape[1],1)*cp.transpose(i_E_w).reshape(-1,1,self.data[1].shape[1]) #P*D*D
                  return i_E_w,i_E_w_2,cp.zeros_like(i_E_w_2) #inv_precision

              E_w,E_w_2,inv_precision=initialization()
              last_run=[E_w,E_w_2,inv_precision,self.p_probability,self.p_wishart]
              self.entire_data=self.data

              entire_order = list(range(self.entire_data[0].shape[0]))
              random.shuffle(entire_order)
              rank = entire_order[-subset:]
              del entire_order[-subset:]

              #rank=random.sample(range(0,self.entire_data[0].shape[0]),sub_set)
              self.data=[self.entire_data[0][rank],self.entire_data[1][rank]]
              self.Q=self.quantity(self.data)

              e_moment,last_run=self.variational_inference(inner_iter,last_run,False)

            else:

              rank = entire_order[-subset:]
              del entire_order[-subset:]
              if len(entire_order)<subset:
                    entire_order = list(range(self.entire_data[0].shape[0]))

              self.data=[self.entire_data[0][rank],self.entire_data[1][rank]]
              self.Q=self.quantity(self.data)

              e_moment,last_run=self.variational_inference(inner_iter,last_run,False)
            Stop=self.early_stop(e_moment[0],i,1e-5,activate=True)

            if Stop:
                return e_moment,last_run
            if i==iter_s-1:
                return e_moment,last_run


    def VIEM(self,max_iter,inner_iter):
      for i in range(max_iter):
        if i ==0:
          e_moment,last_run=self.variational_inference(inner_iter,[])
        else:
          e_moment,last_run=self.variational_inference(inner_iter,last_run)

      return e_moment

    def early_stop(self,E_z,iteration,threshold,activate=True):

        if activate:
            if iteration<5:
                if iteration==0:
                    global moving_container
                    moving_container=cp.expand_dims(E_z,axis=0)
                else:
                    moving_container=cp.concatenate([moving_container,cp.expand_dims(E_z,axis=0)],axis=0)
            else:
                MV=cp.mean(moving_container,axis=0)
                if cp.sum(MV-E_z)<threshold:
                    print('Converged')
                    return True

                else:
                    moving_container=cp.concatenate([moving_container,cp.expand_dims(E_z,axis=0)],axis=0)[1:]
        else:
            pass


def construct_prior(data,sampling=True):
    X,Y=data
    N,P=X.shape
    D=Y.shape[1]

    p=cp.repeat(cp.array([[0.5]]),P,axis=0)
    v_0=D
    V_0=cp.identity(D)

    p_lambda2=cp.sum(cp.var(Y,axis=0))*cp.identity(D)/(D*10*N)
    if cp.log(cp.array(N))>=cp.array((P*D)**2.1/(100.*N)):
        argmax=cp.log(cp.array(N))
    else:
        argmax=cp.array(((P*D)**2.1)/(100.*N))

    p_lambda1=(cp.sum(cp.var(Y,axis=0))*argmax/D)*cp.identity(D)


#     p_lambda1=5*cp.identity(D)
#     p_lambda2=0.05*cp.identity(D)
    if sampling:
        p_lambda1=5*cp.identity(D)
        p_lambda2=0.2*cp.identity(D)
    else:
        p_lambda1=5*cp.identity(D)
        p_lambda2=0.05*cp.identity(D)
    #   print(1/p_lambda1)
    #   print(1/p_lambda2)

    return [[cp.linalg.inv(p_lambda1),cp.linalg.inv(p_lambda2)],[v_0,V_0],p]

In [6]:
# ------------------- Simulation Studies using LASSO -----------------
rng = np.random.default_rng(SEED)
all_metrics = []

for run in range(NUM_RUNS):
    # 1) simulate X and y
    X = make_block_ar1_X(N, P, block=BLOCK, rho=RHO, rng=rng)
    y, beta_true, support = simulate_linear(
        X, s=S, snr=SNR, random_200=RANDOM_200, rng=rng
    )

    # 2) variable selection via LASSO
    sel_idx, coef = fit_lasso_select(X, y, seed=SEED + run)

    # 3) metrics
    m = compute_tpr_fpr(sel_idx, support, P)
    all_metrics.append(m)

    print(f"[run {run+1}/{NUM_RUNS}] selected={m.selected} | TP={m.tp} FP={m.fp} "
          f"FN={m.fn} TN={m.tn} | TPR={m.tpr:.3f} FPR={m.fpr:.3f}")

# 4) report averages
avg_tpr = np.mean([m.tpr for m in all_metrics])
avg_fpr = np.mean([m.fpr for m in all_metrics])
avg_sel = np.mean([m.selected for m in all_metrics])
print("\n== Averages over runs ==")
print(f"avg selected: {avg_sel:.1f} | avg TPR: {avg_tpr:.3f} | avg FPR: {avg_fpr:.3f}")

[run 1/5] selected=217 | TP=113 FP=104 FN=87 TN=696 | TPR=0.565 FPR=0.130
[run 2/5] selected=254 | TP=107 FP=147 FN=93 TN=653 | TPR=0.535 FPR=0.184
[run 3/5] selected=183 | TP=95 FP=88 FN=105 TN=712 | TPR=0.475 FPR=0.110
[run 4/5] selected=221 | TP=105 FP=116 FN=95 TN=684 | TPR=0.525 FPR=0.145
[run 5/5] selected=251 | TP=117 FP=134 FN=83 TN=666 | TPR=0.585 FPR=0.168

== Averages over runs ==
avg selected: 225.2 | avg TPR: 0.537 | avg FPR: 0.147


In [7]:
#Simulations Studies using our methods
rng = np.random.default_rng(SEED)
all_metrics = []

for run in range(5):
    # 1) simulate X and y
    X = make_block_ar1_X(N, P, block=BLOCK, rho=RHO, rng=rng)
    y, beta_true, support = simulate_linear(
        X, s=S, snr=SNR, random_200=RANDOM_200, rng=rng
    )

    # 2) variable selection via our method
    prior = construct_prior([cp.array(X),cp.array(y.reshape(-1,1))])
    e_moment = fixed_variable_selection([cp.array(X),cp.array(y.reshape(-1,1))], prior).VIEM(30,1)
    sel_idx = np.flatnonzero(cp.asnumpy(e_moment[0][:, 0]) > 0.99)

    # 3) metrics
    m = compute_tpr_fpr(sel_idx, support, P)
    all_metrics.append(m)

    print(f"[run {run+1}/{NUM_RUNS}] selected={m.selected} | TP={m.tp} FP={m.fp} "
          f"FN={m.fn} TN={m.tn} | TPR={m.tpr:.3f} FPR={m.fpr:.3f}")

# 4) report averages
avg_tpr = np.mean([m.tpr for m in all_metrics])
avg_fpr = np.mean([m.fpr for m in all_metrics])
avg_sel = np.mean([m.selected for m in all_metrics])
print("\n== Averages over runs ==")
print(f"avg selected: {avg_sel:.1f} | avg TPR: {avg_tpr:.3f} | avg FPR: {avg_fpr:.3f}")

[run 1/5] selected=227 | TP=101 FP=126 FN=99 TN=674 | TPR=0.505 FPR=0.158
[run 2/5] selected=171 | TP=99 FP=72 FN=101 TN=728 | TPR=0.495 FPR=0.090
[run 3/5] selected=185 | TP=123 FP=62 FN=77 TN=738 | TPR=0.615 FPR=0.077
[run 4/5] selected=134 | TP=96 FP=38 FN=104 TN=762 | TPR=0.480 FPR=0.048
[run 5/5] selected=139 | TP=98 FP=41 FN=102 TN=759 | TPR=0.490 FPR=0.051

== Averages over runs ==
avg selected: 171.2 | avg TPR: 0.517 | avg FPR: 0.085
